# Clustering with scikit-learn

We want to explore how to use scikit-learn for clustering. This includes preparing the data, training the model, and evaluating and visualizing the results.

We will use the following modules for this:
* [sklearn.datasets](https://scikit-learn.org/stable/api/sklearn.datasets.html): Tools for using common datasets for ML or for generating synthetic data
* [sklearn.model_selection](https://scikit-learn.org/stable/api/sklearn.model_selection.html): Tools for data splitting, cross-validation, and parameter tuning
* [sklearn.cluster](https://scikit-learn.org/stable/api/sklearn.cluster.html): Collection of clustering models
* [sklearn.metrics](https://scikit-learn.org/stable/api/sklearn.metrics.html): Collection of various metrics for model evaluation

**In general, also refer to the comprehensive [User Guide on clustering](https://scikit-learn.org/stable/modules/clustering.html) from scikit-learn.**

In [ ]:
# Just in case we need help
# Import bia-bob as a helpful Python & Medical AI expert
from bia_bob import bob
import os

bob.initialize(
    endpoint='https://kiara.sc.uni-leipzig.de/api/v1', 
    model="vllm-llama-4-scout-17b-16e-instruct",
    system_prompt=os.getenv('SYSTEM_PROMPT_MEDICAL_AI')
)

In [ ]:
%bob Who are you ? Just 1 sentence!

## Clustering on synthetic data

### Data preparation

For a start and to show the concepts of clustering with scikit-learn, we generate synthetic data for a simple clustering problem.  
We use [`make_blobs`](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html):
* To get 100 data points (samples) with 5 features each in `X`,
* With data points divided in 4 classes in `y`
* And an increased standard deviation in the classes for a more scattered distribution

In [ ]:
from sklearn.datasets import make_blobs

# We use the established notation: X for Features, y for Target
X, y = make_blobs(
    n_samples=100,
    n_features=5,
    centers=4,
    cluster_std=3,
    random_state=42
)

In [ ]:
print("Type X:", type(X))
print("Shape X:", X.shape)
print("First X:", X[0])
print("Type y:", type(y))
print("Shape y:", y.shape)
print("First y:", y[0])

To make the data a little more convenient to work with, we load it into a pandas dataframe, and use pandas' methods to get a better overview on the data.

In [ ]:
import pandas as pd

df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(0, X.shape[1])])
df['target'] = y

In [ ]:
df.info()

In [ ]:
df.describe().T

Visualizing 5-dimensional data (5 features) is not really possible.  
So let's visualize just 2 of the features (2 dimensions of the data) vs. the target in a 2D scatterplot.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.scatterplot(data=df, x=df.columns[0], y=df.columns[1], hue='target')
plt.title(f'Scatterplot of {df.columns[0]} & {df.columns[1]} vs target')
plt.show()

#### Data Scaling

As we can see from the descriptive statistics above, the features have different means, standard deviations, and numeric ranges. Distance-based clustering algorithms, like KMeans, compute distances between data points. Features with larger numeric ranges can dominate the distance measure and disproportionately influence the clustering result, even if they are not inherently more important. Scaling ensures that all features contribute more equally.

Depending on the data, there are various methods for doing this, like described in the [User Guide on data processing](https://scikit-learn.org/stable/modules/preprocessing.html), or in the [Examples on preprocessing](https://scikit-learn.org/stable/auto_examples/preprocessing/index.html).

We will use the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html), which works well for data without outliers, and scales the features to have zero mean and unit variance.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

df_scaled = pd.DataFrame(X_scaled, columns=[f'feature_{i}' for i in range(0, X.shape[1])])
df_scaled['target'] = y

df_scaled.describe().T

## Select and train a model

We want to make use of the basic KMeans clustering algorithm.  
For clustering, we do not necessarely need to split our data for training and test.  

* We import the model [`KMeans`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) and initialize the corresponding Python object using `()` **while also setting specific model parameters**
* Model training is started using the method [`.fit_predict()`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans.fit_predict)
* We pass the data as a whole to this method
* The model parameters are now adjusted to predict the target (clusters) as accurately as possible based on the associated feature

In [ ]:
from sklearn.cluster import KMeans

# Initialize model with model parameters
# - n_clusters: number of expected clusters 
#   -> since we do not know any better yet, we assume it may be 2
model = KMeans(
    n_clusters=2
)

In [ ]:
# Unsupervised training 
# - “Fitting” the model to the training data without known labels
# - return the estimated cluster assignments for the samples
y_pred = model.fit_predict(X_scaled)

### Evaluation

We now want to check how well the model can predict the actual clusters for our data by comparing the predictions with the known classes derive metrics for evaluating the model quality.

* Suitable clustering metrics are described, e.g., in the [User Guide - Clustering Evaluation](https://scikit-learn.org/stable/modules/clustering.html#clustering-evaluation)
* We choose
  * [Silhouette Coefficient](https://scikit-learn.org/stable/modules/clustering.html#silhouette-coefficient):
    * The score is bounded between -1 for incorrect clustering and +1 for highly dense clustering. Scores around zero indicate overlapping clusters.
    * The score is higher when clusters are dense and well separated, which relates to a standard concept of a cluster.
  * [Homogeneity-Completeness measures](https://scikit-learn.org/stable/modules/clustering.html#homogeneity-completeness-and-v-measure):
    * Homogeneity and Completeness are bounded below by 0.0 and above by 1.0 (higher is better)
    * V-measure is the harmonic mean of both
* In general, it makes sense to calculate several metrics in order to get a better impression of the model quality

In [ ]:
from sklearn.metrics import homogeneity_completeness_v_measure, silhouette_score

sc = silhouette_score(X_scaled, y_pred, metric='euclidean')
hcv = homogeneity_completeness_v_measure(y, y_pred)

print('silhouette_score: ', sc)
print('homogeneity_completeness_v_measure: ', hcv)

## Parameter tuning

We did run the KMeans clustering under the assumption there may be 2 clusters. For KMeans, this value is a model parameter. And we can try other values to see whether the metrics for evaluation, i.e., the clustering quality may improve.

### Exercise

Now it's your turn! Check for different values for the parameter `n_clusters` to see if the values of the evaluation metrics can be improved. The goal is to find the value for `n_clusters` that achieves the best evaluation result for both `silhouette_score` and `v_measure`. Plotting the evaluation metrics may help to get a better impression.

Hint: a loop is the easiest approach

**You may ask our assistant bia-bob for help.**

In [ ]:
# ToDo: Parameter tuning for KMeans

## Cluster Analysis

Now that we have our original data and the cluster assignments predicted by the clustering algorithm, we can explore the clusters in more detail. For example, we can compute the **mean value of each feature per cluster** to get an idea of how the clusters differ.

Since the row order of `X` (our original data) and `y_pred` (the predicted cluster labels) is the same, we can combine them into a single DataFrame. Then we can use [`groupby`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) to aggregate the feature values **cluster-wise**.

This gives us a simple way to interpret the clustering result by summarizing each cluster with descriptive statistics.

In [ ]:
df_results = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(0, X.shape[1])])
df_results['predicted_cluster'] = y_pred

In [ ]:
# ToDo